# Amplifier Mount Plans: From Profiles to Kernel Configuration

This notebook demonstrates how `amplifier-app-cli` converts profiles and settings into mount plans that the Amplifier kernel understands.

## Overview

The conversion flow follows these steps:

```
User Intent (CLI + flags)
    ↓
Profile Loading (YAML + inheritance)
    ↓
Settings Merge (local/project/global)
    ↓
Compilation → Mount Plan
    ↓
Kernel Validates & Loads
    ↓
Session Ready
```

## Setup: Import Required Libraries

In [ ]:
import json
from pprint import pprint

from amplifier_app_cli.runtime.config import deep_merge
from amplifier_app_cli.runtime.config import expand_env_vars

# Note: These imports work when running from the amplifier-app-cli repository

## Part 1: Understanding Mount Plan Structure

A mount plan is the contract between the application layer and the kernel. It's a plain Python dictionary with these key sections:

- **session**: Core session configuration (orchestrator, context, limits)
- **providers**: LLM backends (Anthropic, OpenAI, etc.)
- **tools**: Agent capabilities (filesystem, bash, web, etc.)
- **hooks**: Observability and lifecycle hooks
- **agents**: Named configuration overlays for delegation

In [ ]:
# Minimal valid mount plan
minimal_mount_plan = {
    "session": {"orchestrator": "loop-basic", "context": "context-simple"},
    "providers": [{"module": "provider-mock"}],
}

print("Minimal Mount Plan:")
pprint(minimal_mount_plan)

## Part 2: A More Complete Mount Plan

Let's build a more realistic development mount plan with multiple modules and configuration.

In [ ]:
# Development mount plan with full configuration
dev_mount_plan = {
    "session": {
        "orchestrator": "loop-streaming",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
        "injection_budget_per_turn": 10000,
        "injection_size_limit": 10240,
    },
    "context": {"config": {"max_tokens": 200000, "compact_threshold": 0.92, "auto_compact": True}},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "api_key": "${ANTHROPIC_API_KEY}", "debug": True},
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            "config": {"allowed_paths": ["."], "require_approval": False},
        },
        {"module": "tool-bash", "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"},
    ],
    "hooks": [
        {
            "module": "hooks-logging",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-logging@main",
            "config": {"output_dir": ".amplifier/logs", "mode": "session-only"},
        }
    ],
}

print("Development Mount Plan:")
print(json.dumps(dev_mount_plan, indent=2))

## Part 3: Deep Merging - How Profiles Combine

When profiles extend each other (inheritance), or when settings override profile values, Amplifier uses **deep merging** to combine configurations intelligently.

### Deep Merge Strategy

- **Module lists** (providers, tools, hooks): Merged by module ID
- **Config objects**: Recursively merged
- **Primitives**: Child value overrides parent

In [ ]:
# Parent profile configuration
parent_config = {
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "temperature": 0.7, "max_tokens": 4096},
        }
    ]
}

# Child profile configuration (extends parent)
child_config = {
    "providers": [
        {
            "module": "provider-anthropic",
            "config": {
                "debug": True,
                "temperature": 0.9,  # Override parent's temperature
            },
        }
    ]
}

print("Parent Config:")
pprint(parent_config)
print("\nChild Config:")
pprint(child_config)

# Merge them using deep_merge
merged_config = deep_merge(parent_config, child_config)

print("\nMerged Config:")
pprint(merged_config)
print("\nNotice how:")
print("  ✓ 'source' was inherited from parent")
print("  ✓ 'model' and 'max_tokens' were inherited from parent")
print("  ✓ 'temperature' was overridden by child (0.9)")
print("  ✓ 'debug' was added by child")

## Part 4: Environment Variable Expansion

Mount plans support environment variable references using `${VAR_NAME}` syntax, with optional defaults using `${VAR_NAME:default}`.

In [ ]:
import os

# Set some example environment variables
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-example-key-123"
os.environ["LOG_LEVEL"] = "debug"

# Mount plan with environment variable references
config_with_envvars = {
    "providers": [
        {
            "module": "provider-anthropic",
            "config": {
                "api_key": "${ANTHROPIC_API_KEY}",  # Will be replaced
                "debug": "${DEBUG_MODE:false}",  # Will use default "false"
                "log_level": "${LOG_LEVEL}",  # Will be replaced with "debug"
            },
        }
    ]
}

print("Before expansion:")
pprint(config_with_envvars)

# Expand environment variables
expanded_config = expand_env_vars(config_with_envvars)

print("\nAfter expansion:")
pprint(expanded_config)

## Part 5: Adding Multiple Modules

Let's demonstrate how base configuration and overlays combine when adding multiple tools and providers.

In [ ]:
# Base configuration with basic tools
base_config = {
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
        },
        {"module": "tool-bash", "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"},
    ]
}

# Overlay adding web tool and modifying filesystem config
overlay_config = {
    "tools": [
        {"module": "tool-filesystem", "config": {"allowed_paths": ["/home/user/projects"], "require_approval": True}},
        {
            "module": "tool-web",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main",
            "config": {"timeout": 30},
        },
    ]
}

print("Base Configuration:")
pprint(base_config)
print("\nOverlay Configuration:")
pprint(overlay_config)

# Merge them
final_config = deep_merge(base_config, overlay_config)

print("\nFinal Merged Configuration:")
pprint(final_config)
print("\nNotice how:")
print("  ✓ tool-filesystem kept its source from base")
print("  ✓ tool-filesystem gained config from overlay")
print("  ✓ tool-bash remained unchanged")
print("  ✓ tool-web was added from overlay")

## Part 6: Agent Delegation Configuration

The `agents` section is special - it contains named configuration overlays for spawning specialized child sessions, not modules to load immediately.

When the main agent wants to delegate work, it uses these agent configurations to spawn a child session with customized capabilities.

In [ ]:
# Mount plan with agent delegation support
delegation_mount_plan = {
    "session": {"orchestrator": "loop-streaming", "context": "context-simple"},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5"},
        }
    ],
    "tools": [
        {"module": "tool-filesystem"},
        {"module": "tool-bash"},
        {"module": "tool-task"},  # This enables delegation!
    ],
    "agents": {
        # Specialized debugging agent with different model
        "bug-hunter": {
            "description": "Specialized debugging agent",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "config": {
                        "model": "claude-opus-4-1",  # Upgrade to Opus
                        "temperature": 0.3,  # Lower for precision
                    },
                }
            ],
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-bash"},
                # Note: no tool-task (prevent recursive delegation)
            ],
            "system": {"instruction": "You are a debugging specialist. Focus on finding root causes."},
        },
        # Fast research agent with limited tools
        "researcher": {
            "description": "Fast research and information gathering",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "config": {
                        "model": "claude-haiku-4-1",  # Downgrade for speed
                        "temperature": 0.5,
                    },
                }
            ],
            "tools": [
                {"module": "tool-web"},
                {"module": "tool-filesystem"},  # Read-only effectively
            ],
            "system": {"instruction": "You are a research specialist. Gather information quickly and concisely."},
        },
    },
}

print("Mount Plan with Agent Delegation:")
print(json.dumps(delegation_mount_plan, indent=2))

print("\n" + "=" * 60)
print("Key Points:")
print("=" * 60)
print("✓ Main session has tool-task enabled for delegation")
print("✓ 'bug-hunter' agent uses Opus for complex debugging")
print("✓ 'researcher' agent uses Haiku for fast information gathering")
print("✓ Each agent has custom system instructions")
print("✓ Agents can't recursively delegate (no tool-task)")

## Part 7: The Complete Flow - CLI to Mount Plan

Let's simulate the complete flow from CLI invocation to mount plan, as it happens in `amplifier-app-cli`.

This demonstrates the full `resolve_app_config` function behavior:

1. Start with defaults
2. Apply profile configuration
3. Merge settings overlays
4. Apply CLI overrides
5. Expand environment variables

In [ ]:
# Step 1: Default configuration
default_config = {
    "session": {
        "orchestrator": "loop-basic",
        "context": "context-simple",
    },
    "providers": [],
    "tools": [],
    "hooks": [],
}

print("Step 1 - Default Config:")
pprint(default_config)

# Step 2: Profile configuration (simulating loaded profile)
profile_config = {
    "session": {
        "orchestrator": "loop-streaming",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
    },
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5"},
        }
    ],
    "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}],
}

config = deep_merge(default_config, profile_config)
print("\nStep 2 - After Profile Merge:")
pprint(config)

# Step 3: Settings overlay (from user settings)
settings_overlay = {
    "tools": [{"module": "tool-web", "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main"}],
    "hooks": [{"module": "hooks-logging", "config": {"mode": "all"}}],
}

config = deep_merge(config, settings_overlay)
print("\nStep 3 - After Settings Merge:")
pprint(config)

# Step 4: CLI overrides (from command line flags like --model)
cli_overrides = {
    "providers": [
        {
            "module": "provider-anthropic",
            "config": {
                "model": "claude-opus-4-1",  # Override model via --model flag
                "api_key": "${ANTHROPIC_API_KEY}",
            },
        }
    ]
}

config = deep_merge(config, cli_overrides)
print("\nStep 4 - After CLI Overrides:")
pprint(config)

# Step 5: Expand environment variables
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-api-key-from-env"
final_mount_plan = expand_env_vars(config)

print("\nStep 5 - Final Mount Plan (after env expansion):")
print(json.dumps(final_mount_plan, indent=2))

print("\n" + "=" * 60)
print("Compilation Summary:")
print("=" * 60)
print("✓ Orchestrator upgraded from 'loop-basic' to 'loop-streaming'")
print("✓ Context upgraded from 'context-simple' to 'context-persistent'")
print("✓ Provider added from profile")
print("✓ Model overridden via CLI: 'claude-opus-4-1'")
print("✓ Tools combined: filesystem, bash, web")
print("✓ Hooks added from settings")
print("✓ API key resolved from environment")

## Part 8: Production Configuration Best Practices

Let's create a production-ready mount plan with safety controls and cost management.

In [ ]:
production_mount_plan = {
    "session": {
        "orchestrator": "loop-events",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-events@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
        # Conservative injection limits for cost control
        "injection_budget_per_turn": 500,
        "injection_size_limit": 8192,
    },
    "context": {
        "config": {
            "max_tokens": 200000,
            "compact_threshold": 0.95,  # More aggressive compaction
            "auto_compact": True,
        }
    },
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {
                "model": "claude-sonnet-4-5",
                "api_key": "${ANTHROPIC_API_KEY}",
                "max_tokens": 4096,  # Cap output length
                "debug": False,  # No debug logs in production
            },
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            "config": {
                "allowed_paths": ["/app/data"],  # Restricted paths
                "require_approval": True,  # Human approval required
            },
        }
    ],
    "hooks": [
        {
            "module": "hooks-logging",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-logging@main",
            "config": {
                "mode": "all",
                "redaction_enabled": True,  # Redact sensitive data
            },
        },
        {"module": "hooks-backup", "source": "git+https://github.com/microsoft/amplifier-module-hooks-backup@main"},
    ],
}

print("Production Mount Plan with Safety Controls:")
print(json.dumps(production_mount_plan, indent=2))

print("\n" + "=" * 60)
print("Production Safety Features:")
print("=" * 60)
print("✓ Conservative injection limits (500 tokens/turn, 8KB max)")
print("✓ Aggressive context compaction (0.95 threshold)")
print("✓ Output token cap (4096 max)")
print("✓ Restricted filesystem access (/app/data only)")
print("✓ Human approval required for file operations")
print("✓ Comprehensive logging with redaction")
print("✓ Automatic session backup")
print("✓ Debug mode disabled")

## Summary

This notebook demonstrated:

1. **Mount Plan Structure**: The kernel's contract format
2. **Deep Merging**: How profiles and settings intelligently combine
3. **Environment Variables**: Dynamic configuration resolution
4. **Module Management**: Adding, overriding, and configuring modules
5. **Agent Delegation**: Specialized child session configurations
6. **Complete Flow**: From CLI to final mount plan
7. **Production Best Practices**: Safety and cost controls

### Key Takeaways

- **Mount plans** are the single format the kernel understands
- **Profiles** compile to mount plans through inheritance and merging
- **Deep merging** preserves sources and combines configs intelligently
- **Environment variables** enable dynamic, secure configuration
- **Agent sections** are overlays for delegation, not immediate loads
- **Production configs** should include safety limits and approval gates

### Related Documentation

- [Profiles Guide](./guides/profiles.md)
- [Modules Guide](./guides/modules.md)
- [CLI Guide](./guides/cli.md)
- [Mount Plans Guide](./guides/mounts.md)